# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
weather_df = pd.read_csv("output_data/cities.csv", encoding = 'latin-1')

weather_df

# Encountered an error where there was an invalid UTC-8 byte in the data. I found a suggestion to try encoding it as 'latin-1'
# from this website: https://bobbyhadz.com/blog/python-unicodedecodeerror-utf-8-codec-cant-decode-byte

,City Name,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Linxia Chengguanzhen,35.6003,103.2064,69.78,81,96,3.87,CN,1660339551
1,Rikitea,-23.1203,-134.9692,73.18,73,100,23.51,PF,1660339554
2,Horsham,51.0640,-0.3272,76.05,52,0,5.75,GB,1660339557
3,Port Alfred,-33.5906,26.8910,59.13,72,98,15.43,ZA,1660339559
4,Nome,64.5011,-165.4064,51.87,87,100,9.22,US,1660339561
...,...,...,...,...,...,...,...,...,...
504,Graaff-Reinet,-32.2522,24.5308,50.92,74,52,2.98,ZA,1660340624
505,San Quintín,30.4833,-115.9500,76.95,70,13,12.44,MX,1660340625
506,Antofagasta,-23.6500,-70.4000,58.28,80,58,9.40,CL,1660340437
507,Hambantota,6.1241,81.1185,77.34,94,20,2.30,LK,1660340629


In [96]:
weather_df.rename(columns = {"City Name": "City", "Long": "Lng"}, inplace = True)

weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Linxia Chengguanzhen,35.6003,103.2064,69.78,81,96,3.87,CN,1660339551
1,Rikitea,-23.1203,-134.9692,73.18,73,100,23.51,PF,1660339554
2,Horsham,51.0640,-0.3272,76.05,52,0,5.75,GB,1660339557
3,Port Alfred,-33.5906,26.8910,59.13,72,98,15.43,ZA,1660339559
4,Nome,64.5011,-165.4064,51.87,87,100,9.22,US,1660339561
...,...,...,...,...,...,...,...,...,...
504,Graaff-Reinet,-32.2522,24.5308,50.92,74,52,2.98,ZA,1660340624
505,San Quintín,30.4833,-115.9500,76.95,70,13,12.44,MX,1660340625
506,Antofagasta,-23.6500,-70.4000,58.28,80,58,9.40,CL,1660340437
507,Hambantota,6.1241,81.1185,77.34,94,20,2.30,LK,1660340629


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Adding my API key to gmaps

gmaps.configure(api_key=g_key)

In [9]:
# Creating locations for API

locations = weather_df[["Lat", "Long"]].astype(float)

In [11]:
# Creating humidity for API

humidity = weather_df["Humidity"].astype(float)

0      81.0
1      73.0
2      52.0
3      72.0
4      87.0
       ... 
504    74.0
505    70.0
506    80.0
507    94.0
508    94.0
Name: Humidity, Length: 509, dtype: float64

In [14]:
# Create map

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating = False, max_intensity = 100, point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [97]:
warm_enough_df = weather_df.loc[weather_df["Max Temp"] > 65]

good_temp = warm_enough_df.loc[warm_enough_df["Max Temp"] < 80]

good_temp_low_humid = good_temp.loc[good_temp["Humidity"] <= 40]

good_temp_low_humid_low_clouds = good_temp_low_humid.loc[good_temp_low_humid["Cloudiness"] <= 30]

good_temp_low_humid_low_clouds_breeze = good_temp_low_humid_low_clouds.loc[good_temp_low_humid_low_clouds["Wind Speed"] < 7]

good_temp_low_humid_low_clouds_breeze

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
31,Norman Wells,65.2820,-126.8329,75.24,29,20,4.61,CA,1660339613
160,Aklavik,68.2191,-135.0107,74.66,32,5,5.44,CA,1660339886
175,Yangi Marg`ilon,40.4272,71.7189,79.52,31,0,2.30,UZ,1660339917
244,Matagami,49.7502,-77.6328,69.91,37,0,6.91,CA,1660340062
287,Camapuã,-19.5314,-54.0439,69.21,30,0,1.39,BR,1660340155
425,Vertou,47.1687,-1.4693,78.78,34,0,2.30,FR,1660340450
444,São Domingos,-13.3983,-46.3183,78.53,26,25,5.46,BR,1660340491
490,Maniwaki,46.3834,-75.9660,71.98,37,2,6.91,CA,1660340593
503,La Rioja,42.2500,-2.5000,73.18,38,0,5.88,ES,1660340622


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [98]:

hotel_df = good_temp_low_humid_low_clouds_breeze.copy()

hotel_df["Hotel Name"] = ""

hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
31,Norman Wells,65.2820,-126.8329,75.24,29,20,4.61,CA,1660339613,
160,Aklavik,68.2191,-135.0107,74.66,32,5,5.44,CA,1660339886,
175,Yangi Marg`ilon,40.4272,71.7189,79.52,31,0,2.30,UZ,1660339917,
244,Matagami,49.7502,-77.6328,69.91,37,0,6.91,CA,1660340062,
287,Camapuã,-19.5314,-54.0439,69.21,30,0,1.39,BR,1660340155,
425,Vertou,47.1687,-1.4693,78.78,34,0,2.30,FR,1660340450,
444,São Domingos,-13.3983,-46.3183,78.53,26,25,5.46,BR,1660340491,
490,Maniwaki,46.3834,-75.9660,71.98,37,2,6.91,CA,1660340593,
503,La Rioja,42.2500,-2.5000,73.18,38,0,5.88,ES,1660340622,


In [100]:
# Construct URL

lats = list(hotel_df["Lat"])

longs = list(hotel_df["Lng"])

base_url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json?"

for index in range(0,len(lats)):
    lat = lats[index]
    long = longs[index]

    query = f"input=hotel&inputtype=textquery&locationbias=circle:5000@{lat},{long}&fields=name&key="
    
    full_url = base_url + query + g_key
    
    response = requests.get(full_url).json()

    hotel_df.iloc[index,9] = response['candidates'][0]['name']

hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
31,Norman Wells,65.2820,-126.8329,75.24,29,20,4.61,CA,1660339613,Heritage Hotel
160,Aklavik,68.2191,-135.0107,74.66,32,5,5.44,CA,1660339886,Sundog Inn
175,Yangi Marg`ilon,40.4272,71.7189,79.52,31,0,2.30,UZ,1660339917,TANTANA HOTEL FERGANA
244,Matagami,49.7502,-77.6328,69.91,37,0,6.91,CA,1660340062,Motel Le Caribou
287,Camapuã,-19.5314,-54.0439,69.21,30,0,1.39,BR,1660340155,COLLINA HOTEL
425,Vertou,47.1687,-1.4693,78.78,34,0,2.30,FR,1660340450,ibis budget Nantes Rezé Aéroport
444,São Domingos,-13.3983,-46.3183,78.53,26,25,5.46,BR,1660340491,Pousada Solar Sunflowers.
490,Maniwaki,46.3834,-75.9660,71.98,37,2,6.91,CA,1660340593,Centre Chateau Logue
503,La Rioja,42.2500,-2.5000,73.18,38,0,5.88,ES,1660340622,"Albergue ""Las Huellas"""


In [101]:
# Make markers for map

marker_list = []

for index in range(0,9):
    lat = hotel_df.iloc[index,1]
    long = hotel_df.iloc[index,2]
    marker = (lat, long)
    
    marker_list.append(marker)
    
marker_list
    

[(65.282, -126.8329),
 (68.2191, -135.0107),
 (40.4272, 71.7189),
 (49.7502, -77.6328),
 (-19.5314, -54.0439),
 (47.1687, -1.4693),
 (-13.3983, -46.3183),
 (46.3834, -75.966),
 (42.25, -2.5)]

In [102]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [103]:
# Add marker layer ontop of heat map

marker_layer = gmaps.marker_layer(marker_list, info_box_content = hotel_info)

# Display figure

fig1 = gmaps.figure()
fig1.add_layer(heat_layer)
fig1.add_layer(marker_layer)
fig1

Figure(layout=FigureLayout(height='420px'))